# Experiment SetUp - 50 Trials

Parameters:
- l1
- l1 vs. batch_size
- epochs vs. batch_size
- epochs vs. lr
- lr vs. batch_size

### Imports

In [1]:
from functools import partial
import os
import random 

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import random_split

from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.basic_variant import BasicVariantGenerator
from ray.tune import Callback

In [2]:
torch.manual_seed(40)
random.seed(40)
np.random.seed(40)

#### Class to print chosen l1 param

In [3]:
class PrintChosenL1Callback(Callback):
    def on_trial_complete(self, iteration, trials, trial, **info):
        chosen_l1 = trial.config["l1"]
        print(f"Trial {trial.trial_id}: Chosen l1 = {chosen_l1}")

In [4]:
print_l1_callback = PrintChosenL1Callback()

### CNN Model SetUp

In [5]:
class FashionCNN(nn.Module):
    def __init__(self, l1=64):
        super(FashionCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 7 * 7, l1)
        self.fc2 = nn.Linear(l1, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Training

#### Data Loading function

In [6]:
def load_data(data_dir="./data"):
    transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))])

    trainset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=True,
        transform=transform)
    testset = torchvision.datasets.FashionMNIST(data_dir,
        download=True,
        train=False,
        transform=transform)

    return trainset, testset

#### Training function

In [7]:
def train_fashion_mnist(config):
    net = FashionCNN(config["l1"]) 

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    # # loading data
    trainset, testset = load_data()
    
    # Split the training set into subsets
    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(trainset, [test_abs, len(trainset) - test_abs])

    # Create data loaders for subsets
    trainloader = torch.utils.data.DataLoader(
        train_subset, batch_size=config["batch_size"], shuffle=True, num_workers=2
    )
    valloader = torch.utils.data.DataLoader(
        val_subset, batch_size=config["batch_size"], shuffle=True, num_workers=2
    )

    # defining the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["learning_rate"])

    for epoch in range(config["epochs"]):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            epoch_steps += 1

            if i % 2000 == 1999:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1, running_loss / epoch_steps))
                running_loss = 0.0

        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        with torch.no_grad():
            for data in valloader:
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_steps += 1

        tune.report(
            mean_accuracy=correct / total,
            mean_val_loss=val_loss / val_steps
        )

#### Experiment Parameters

#### AHSA scheduler

In [8]:
scheduler = ASHAScheduler(
    metric="mean_val_loss",
    mode="min",
    max_t=15,
    grace_period=1,
    reduction_factor=2,
)

### 1st Experiment - l1

In [9]:
num_samples = 1

config = {
    "epochs": 15,
    "learning_rate": 1e-3, # 0.001
    "batch_size": 64,
    "l1": tune.grid_search([32, 64, 128, 256]) # 32, 64, 128
}

In [10]:
load_data()

result_l1 = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 2},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/e1',
    search_alg=BasicVariantGenerator(random_state=40),
    #scheduler=scheduler,
    chdir_to_trial_dir=False,
    callbacks=[print_l1_callback]
)

best_trial = result_l1.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

2023-08-22 09:46:25,720	INFO worker.py:1431 -- Connecting to existing Ray cluster at address: 141.68.100.65:6379...
2023-08-22 09:46:25,739	INFO worker.py:1612 -- Connected to Ray cluster. View the dashboard at 141.68.100.65:8265 
2023-08-22 09:47:02,108	INFO tune.py:226 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2023-08-22 09:47:02,124	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) [2023-08-22 09:47:06,508 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196525568; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 09:47:13,203	WARNING util.py:315 -- The `on_step_begin` operation took 0.789 s, which may be a performance bottleneck.
(raylet) [2023-08-22 09:47:16,509 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196357632; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:47:26,519 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196312576; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:47:36,611 E 1659495 1659513] (raylet) file_system_m

Trial name,mean_accuracy,mean_val_loss
train_fashion_mnist_c2748_00000,0.811167,0.533731
train_fashion_mnist_c2748_00001,0.823333,0.49121
train_fashion_mnist_c2748_00002,0.821833,0.501295
train_fashion_mnist_c2748_00003,0.82125,0.504545


(raylet) [2023-08-22 09:48:06,658 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196181504; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:48:16,668 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196111872; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:48:26,740 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196046336; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:48:36,809 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13196046336; capacity: 49044

Trial c2748_00000: Chosen l1 = 32
Trial c2748_00002: Chosen l1 = 128
Trial c2748_00001: Chosen l1 = 64
Trial c2748_00003: Chosen l1 = 256


2023-08-22 09:58:55,988	INFO tune.py:1148 -- Total run time: 713.86 seconds (713.59 seconds for the tuning loop).


Chosen l1: 64
Best trial config: {'epochs': 15, 'learning_rate': 0.001, 'batch_size': 64, 'l1': 64}
Best trial metrics: {'mean_accuracy': {'max': 0.8233333333333334, 'min': 0.44225, 'avg': 0.7548444444444442, 'last': 0.8233333333333334, 'last-5-avg': 0.8172333333333333, 'last-10-avg': 0.8036000000000001}, 'mean_val_loss': {'max': 2.143372167932226, 'min': 0.491209994922293, 'avg': 0.7605234654342873, 'last': 0.491209994922293, 'last-5-avg': 0.5108064036737098, 'last-10-avg': 0.5463347244294401}, 'time_this_iter_s': {'max': 50.83018946647644, 'min': 41.33314752578735, 'avg': 46.21753977139791, 'last': 44.04406690597534, 'last-5-avg': 46.462007665634154, 'last-10-avg': 46.027286529541016}, 'done': {'max': False, 'min': False, 'avg': 0.0, 'last': False, 'last-5-avg': 0.0, 'last-10-avg': 0.0}, 'training_iteration': {'max': 15, 'min': 1, 'avg': 8.0, 'last': 15, 'last-5-avg': 13.0, 'last-10-avg': 10.5}, 'time_total_s': {'max': 693.2630965709686, 'min': 46.41031455993652, 'avg': 371.854098653

(raylet) [2023-08-22 09:59:00,146 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13194276864; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:59:10,198 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13194244096; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:59:20,261 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13194244096; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 09:59:30,318 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13194240000; capacity: 49044

### 2nd Experiment - l1 vs. batch_size

In [11]:
num_samples = 1

config = {
    "epochs": 15,
    "learning_rate": 1e-3, 
    "batch_size": tune.grid_search([16, 32, 64, 128]),
    "l1": tune.grid_search([32, 64, 128, 256])
}

(raylet) [2023-08-22 11:02:14,401 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13190115328; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:02:24,470 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13190115328; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:02:34,549 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13190115328; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:02:44,619 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13190111232; capacity: 49044

In [12]:
load_data()

result_l1_bs = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 2},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/e2',
    search_alg=BasicVariantGenerator(random_state=40),
    #scheduler=scheduler,
    chdir_to_trial_dir=False,
    callbacks=[print_l1_callback]
)

best_trial = result_l1_bs.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

2023-08-22 11:02:54,392	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) [2023-08-22 11:02:54,682 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13190033408; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:03:04,773 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189808128; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:03:14,815 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189701632; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:03:24,823 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189664768; capacity: 49044

Trial name,mean_accuracy,mean_val_loss
train_fashion_mnist_71a80_00000,0.866,0.368883
train_fashion_mnist_71a80_00001,0.850917,0.423407
train_fashion_mnist_71a80_00002,0.816667,0.509637
train_fashion_mnist_71a80_00003,0.77525,0.617568
train_fashion_mnist_71a80_00004,0.867167,0.361071
train_fashion_mnist_71a80_00005,0.8525,0.420648
train_fashion_mnist_71a80_00006,0.826833,0.490383
train_fashion_mnist_71a80_00007,0.783,0.596836
train_fashion_mnist_71a80_00008,0.867,0.364469
train_fashion_mnist_71a80_00009,0.852917,0.412745


(raylet) [2023-08-22 11:03:44,987 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189627904; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:03:55,020 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189615616; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:04:05,032 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189738496; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 11:04:15,045 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13189718016; capacity: 49044

(func pid=1784699) [1,  2000] loss: 1.899


...

(raylet) [2023-08-22 12:01:55,303 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180469248; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:02:05,304 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180436480; capacity: 490444898304. Object creation will fail if spilling is required.


Trial 71a80_00012: Chosen l1 = 256


2023-08-22 12:02:13,437	INFO tune.py:1148 -- Total run time: 3559.05 seconds (3558.94 seconds for the tuning loop).


Chosen l1: 64
Best trial config: {'epochs': 15, 'learning_rate': 0.001, 'batch_size': 16, 'l1': 64}
Best trial metrics: {'mean_accuracy': {'max': 0.86825, 'min': 0.7314166666666667, 'avg': 0.831161111111111, 'last': 0.8671666666666666, 'last-5-avg': 0.8638833333333334, 'last-10-avg': 0.8519083333333333}, 'mean_val_loss': {'max': 0.7711916718880335, 'min': 0.3606877371817827, 'avg': 0.4653202255232467, 'last': 0.36107098484039307, 'last-5-avg': 0.3746354199528694, 'last-10-avg': 0.4061342977002263}, 'time_this_iter_s': {'max': 142.6425280570984, 'min': 82.88266825675964, 'avg': 100.65660058657328, 'last': 111.89350271224976, 'last-5-avg': 98.82714576721192, 'last-10-avg': 106.49528141021729}, 'done': {'max': False, 'min': False, 'avg': 0.0, 'last': False, 'last-5-avg': 0.0, 'last-10-avg': 0.0}, 'training_iteration': {'max': 15, 'min': 1, 'avg': 8.0, 'last': 15, 'last-5-avg': 13.0, 'last-10-avg': 10.5}, 'time_total_s': {'max': 1509.8490087985992, 'min': 100.12897157669067, 'avg': 785.088

(raylet) [2023-08-22 12:02:15,323 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180416000; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:02:25,408 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180416000; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:02:35,507 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180407808; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:02:45,606 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180407808; capacity: 49044

### 3rd Experiment - epochs vs. batch_size

In [13]:
num_samples = 1

config = {
    "epochs": tune.grid_search([5, 10, 15, 20]),
    "learning_rate": 1e-3, 
    "batch_size": tune.grid_search([16, 32, 64, 128]),
    "l1": 64
}

(raylet) [2023-08-22 12:05:36,955 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180198912; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:05:47,031 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180190720; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:05:57,123 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180190720; capacity: 490444898304. Object creation will fail if spilling is required.


In [14]:
load_data()

result_ep_bs = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 2},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/e3',
    search_alg=BasicVariantGenerator(random_state=40),
    chdir_to_trial_dir=False,
    #scheduler=scheduler,
    callbacks=[print_l1_callback]
)

best_trial = result_ep_bs.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

2023-08-22 12:06:06,684	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) [2023-08-22 12:06:07,205 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13180059648; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:06:18,702 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13179940864; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 12:06:25,503	WARNING util.py:315 -- The `on_step_begin` operation took 8.614 s, which may be a performance bottleneck.
(raylet) Failed to connect to GCS. Please check `gcs_server.out` for more details.
(raylet) [2023-08-22 12:06:28,791 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13179932672; capacity: 490444898304. Object creation will fail if spilling is re

Trial name,mean_accuracy,mean_val_loss
train_fashion_mnist_4609f_00000,0.83475,0.460747
train_fashion_mnist_4609f_00001,0.801083,0.560654
train_fashion_mnist_4609f_00002,0.761333,0.668506
train_fashion_mnist_4609f_00003,0.673667,1.12252
train_fashion_mnist_4609f_00004,0.861167,0.392781
train_fashion_mnist_4609f_00005,0.831917,0.462848
train_fashion_mnist_4609f_00006,0.783417,0.584845
train_fashion_mnist_4609f_00007,0.76625,0.661754
train_fashion_mnist_4609f_00008,0.870667,0.358491
train_fashion_mnist_4609f_00009,0.852667,0.40603


(raylet) [2023-08-22 12:07:09,303 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13179666432; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:07:19,355 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13179645952; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:07:29,452 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13179600896; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 12:07:39,454 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13179576320; capacity: 49044

(func pid=2671671) [1,  2000] loss: 1.750


...

(raylet) [2023-08-22 13:01:21,695 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171314688; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:01:31,741 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171310592; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:01:41,745 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171290112; capacity: 490444898304. Object creation will fail if spilling is required.


Trial 4609f_00012: Chosen l1 = 64


2023-08-22 13:01:47,622	INFO tune.py:1148 -- Total run time: 3340.94 seconds (3340.74 seconds for the tuning loop).


Chosen l1: 64
Best trial config: {'epochs': 20, 'learning_rate': 0.001, 'batch_size': 16, 'l1': 64}
Best trial metrics: {'mean_accuracy': {'max': 0.8865833333333333, 'min': 0.74825, 'avg': 0.8494374999999996, 'last': 0.8851666666666667, 'last-5-avg': 0.8827666666666666, 'last-10-avg': 0.8773}, 'mean_val_loss': {'max': 0.7231627244551977, 'min': 0.31827534997463225, 'avg': 0.41839380793385206, 'last': 0.318992173401018, 'last-5-avg': 0.32919460942397516, 'last-10-avg': 0.3446958331711591}, 'time_this_iter_s': {'max': 148.1736969947815, 'min': 63.23625326156616, 'avg': 93.16104694604871, 'last': 63.93099665641785, 'last-5-avg': 63.80419611930847, 'last-10-avg': 63.76105201244354}, 'done': {'max': False, 'min': False, 'avg': 0.0, 'last': False, 'last-5-avg': 0.0, 'last-10-avg': 0.0}, 'training_iteration': {'max': 20, 'min': 1, 'avg': 10.5, 'last': 20, 'last-5-avg': 18.0, 'last-10-avg': 15.5}, 'time_total_s': {'max': 1863.2209389209747, 'min': 133.67249202728271, 'avg': 1129.8015766978262,

(raylet) [2023-08-22 13:01:51,808 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171286016; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:02:01,811 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171286016; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:02:11,812 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171257344; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:02:21,851 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13171240960; capacity: 49044

### 4th Experiment - epochs vs. lr

In [15]:
num_samples = 50

config = {
    "epochs": tune.choice([5, 10, 15, 20]),
    "learning_rate": tune.loguniform(1e-4, 1e-1), 
    "batch_size": 64,
    "l1": 64
}

(raylet) [2023-08-22 13:05:42,452 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13170900992; capacity: 490444898304. Object creation will fail if spilling is required.


In [16]:
load_data()

result_ep_lr = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 2},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/e4',
    search_alg=BasicVariantGenerator(random_state=40),
    #scheduler=scheduler,
    chdir_to_trial_dir=False,
    callbacks=[print_l1_callback]
)

best_trial = result_ep_lr.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

2023-08-22 13:05:47,962	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) [2023-08-22 13:06:10,826 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13170388992; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 13:06:15,086	WARNING util.py:315 -- The `on_step_begin` operation took 14.376 s, which may be a performance bottleneck.
(raylet) Failed to connect to GCS. Please check `gcs_server.out` for more details.
(raylet) Failed to publish error: Traceback (most recent call last):
(raylet)   File "/opt/conda/lib/python3.10/site-packages/ray/_private/worker.py", line 2144, in connect
(raylet)     node.check_version_info()
(raylet)   File "/opt/conda/lib/python3.10/site-packages/ray/_private/node.py", line 339, in check_version_info
(raylet)     cluster_metadata = ray_usage_lib.get_cluster_metadata(self.get_gcs_client())
(raylet)   File "/opt/conda/lib/python3.10/site-packages/ray/_private/usage/usage_lib.py", line 710, in get

Trial name,mean_accuracy,mean_val_loss
train_fashion_mnist_9ca59_00000,0.909417,0.25125
train_fashion_mnist_9ca59_00001,0.914,0.248553
train_fashion_mnist_9ca59_00002,0.883583,0.326328
train_fashion_mnist_9ca59_00003,0.905833,0.259207
train_fashion_mnist_9ca59_00004,0.878833,0.348302
train_fashion_mnist_9ca59_00005,0.891083,0.294374
train_fashion_mnist_9ca59_00006,0.8295,0.476954
train_fashion_mnist_9ca59_00007,0.8085,0.535065
train_fashion_mnist_9ca59_00008,0.80525,0.54239
train_fashion_mnist_9ca59_00009,0.715417,0.813952


(raylet) [2023-08-22 13:07:22,744 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13170012160; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:07:32,758 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13170008064; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 13:07:41,951	WARNING util.py:315 -- The `on_step_begin` operation took 0.679 s, which may be a performance bottleneck.
(raylet) [2023-08-22 13:07:42,762 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13169905664; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:07:52,826 E 1659495 1659513] (raylet) file_system_m

Trial 9ca59_00002: Chosen l1 = 64


...

(raylet) [2023-08-22 13:18:57,323 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13167906816; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:19:07,392 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13167833088; capacity: 490444898304. Object creation will fail if spilling is required.
(raylet) [2023-08-22 13:19:17,502 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13167828992; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 13:19:20,396	WARNING util.py:315 -- The `on_step_begin` operation took 7.119 s, which may be a performance bottleneck.
(raylet) [2023-08-22 13:19:27,533 E 1659495 1659513] (raylet) file_system_m

Trial 9ca59_00004: Chosen l1 = 64


(raylet) [2023-08-22 13:23:39,510 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13167005696; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 13:23:46,380	WARNING util.py:315 -- The `on_step_begin` operation took 11.085 s, which may be a performance bottleneck.
(raylet) [2023-08-22 13:23:49,556 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13167005696; capacity: 490444898304. Object creation will fail if spilling is required.
2023-08-22 13:23:58,898	WARNING util.py:315 -- The `on_step_begin` operation took 2.477 s, which may be a performance bottleneck.
(raylet) [2023-08-22 13:23:59,645 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13166977024; capacity: 490444898

Chosen l1: 64
Best trial config: {'epochs': 15, 'learning_rate': 0.076531818780208, 'batch_size': 64, 'l1': 64}
Best trial metrics: {'mean_accuracy': {'max': 0.9173333333333333, 'min': 0.8428333333333333, 'avg': 0.9010499999999998, 'last': 0.914, 'last-5-avg': 0.9154333333333333, 'last-10-avg': 0.9118166666666667}, 'mean_val_loss': {'max': 0.42279110920239005, 'min': 0.2346625860701216, 'avg': 0.27293688767979324, 'last': 0.24855331085780832, 'last-5-avg': 0.23969115179745443, 'last-10-avg': 0.24600376854947906}, 'time_this_iter_s': {'max': 51.808491945266724, 'min': 40.41565012931824, 'avg': 45.80063452720641, 'last': 45.41266918182373, 'last-5-avg': 46.70826668739319, 'last-10-avg': 46.305001449584964}, 'done': {'max': False, 'min': False, 'avg': 0.0, 'last': False, 'last-5-avg': 0.0, 'last-10-avg': 0.0}, 'training_iteration': {'max': 15, 'min': 1, 'avg': 8.0, 'last': 15, 'last-5-avg': 13.0, 'last-10-avg': 10.5}, 'time_total_s': {'max': 687.0095179080963, 'min': 46.11673069000244, 'a

(raylet) [2023-08-22 13:27:00,453 E 1659495 1659513] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-08-22_09-44-45_245297_1659393 is over 95% full, available space: 13166247936; capacity: 490444898304. Object creation will fail if spilling is required.


### 5th Experiment - batch_size vs. lr

In [ ]:
num_samples = 50

config = {
    "epochs": 15,
    "learning_rate": tune.loguniform(1e-4, 1e-1), 
    "batch_size": tune.choice([16, 32, 64, 128]),
    "l1": 64
}

In [ ]:
load_data()

result_bs_lr = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 2},
    config=config,
    num_samples=num_samples,
    storage_path='./tune_runs/e5',
    search_alg=BasicVariantGenerator(random_state=40),
    #scheduler=scheduler,
    chdir_to_trial_dir=False,
    callbacks=[print_l1_callback]
)

best_trial = result_bs_lr.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)

## 6th Experiment - l1, batch_size, lr, epochs - 6h

In [ ]:
config = {
    "epochs": tune.choice([5, 10, 15, 20]),
    "learning_rate": tune.loguniform(1e-4, 1e-1), 
    "batch_size": tune.choice([16, 32, 64, 128]),
    "l1": tune.choice([32, 64, 128, 256])
}

In [ ]:
load_data()

scheduler = ASHAScheduler(
    metric="mean_val_loss",
    mode="min",
    max_t=20,
    grace_period=1,
    reduction_factor=2,
)

result_bs_lr = tune.run(
    partial(train_fashion_mnist),
    resources_per_trial={"cpu": 8, "gpu": 2},
    config=config,
    #num_samples=num_samples,
    storage_path='./tune_runs/e6',
    search_alg=BasicVariantGenerator(random_state=40),
    scheduler=scheduler,
    time_budget_s=300, #21600
    chdir_to_trial_dir=False,
    callbacks=[print_l1_callback]
)

best_trial = result_bs_lr.get_best_trial("mean_val_loss", mode="min")
best_config = best_trial.config
chosen_l1 = best_config["l1"]

print("Chosen l1:", chosen_l1)

best_metrics = best_trial.metric_analysis

print("Best trial config:", best_config)
print("Best trial metrics:", best_metrics)